In [1]:
import pyscf

In [2]:
molecule = """
Cr -1.32078 0.00005 -0.00007
Cr 1.32077 0.00005 -0.00007
O 0.00000 -0.16583 1.45468
O 0.00000 1.34277 -0.58372
O 0.00000 -1.17683 -0.87101
H 0.00002 0.50128 2.15993
H 0.00056 1.61869 -1.51448
H -0.00044 -2.12079 -0.64413
N -2.64980 -1.44569 0.71142
H -2.18696 -2.18198 1.24440
H -3.05396 -1.84420 -0.13607
H -3.36727 -1.00512 1.28721
N -2.64980 1.33902 0.89630
N -2.64980 0.10677 -1.60777
H -3.36727 -0.61216 -1.51411
H -3.05396 0.80432 1.66516
N 2.64980 -1.44568 0.71142
N 2.64979 1.33903 0.89630
N 2.64980 0.10678 -1.60777
H -2.18697 2.16873 1.26745
H -3.36727 1.61737 0.22686
H -2.18696 0.01334 -2.51190
H -3.05397 1.03998 -1.52914
H 2.18696 -2.18197 1.24440
H 3.05396 -1.84419 -0.13608
H 3.36727 -1.00510 1.28720
H 2.18695 2.16874 1.26745
H 3.05396 0.80433 1.66516
H 3.36726 1.61738 0.22685
H 2.18696 0.01335 -2.51190
H 3.05396 1.03999 -1.52914
H 3.36727 -0.61215 -1.51411
"""

In [3]:
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   6,
        charge  =   3,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()

symmetry:  C1


******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 63  num. singly occ = 6
init E= -2649.46926360849
  HOMO = -0.192719824232853  LUMO = -0.0859499418977088
cycle= 1 E= -2648.33372169409  delta_E= 1.14  |g|= 1.13  |ddm|= 3.97
  HOMO = -0.475198979599963  LUMO = -0.293329515685072
cycle= 2 E= -2648.74406399993  delta_E= -0.41  |g|= 0.627  |ddm|= 1.22
  HOMO = -0.586114631333281  LUMO = -0.293446133720431
cycle= 3 E= -2648.82815509457  delta_E= -0.0841  |g|= 0.167  |ddm|= 0.545
  HOMO = -0.590572205613915  LUMO = -0.292500265426309
cycle= 4 E= -2648.83694212617  delta_E= -0.00879  |g|= 0.0381  |ddm|= 0.224
  HOMO = -0.591633

((array([1.99999824e+00, 1.99999458e+00, 1.99960088e+00, 4.14818565e-01,
         3.75285572e-03, 1.99998869e+00, 1.99999390e+00, 1.99999390e+00,
         1.99864797e+00, 1.99892193e+00, 1.99892191e+00, 1.28675457e-02,
         1.59266417e-02, 1.59291528e-02, 5.77960125e-01, 8.23786075e-01,
         8.72312122e-01, 5.77942887e-01, 9.69447199e-01, 6.88437541e-03,
         1.27261431e-02, 1.29838516e-02, 6.88399354e-03, 1.35033745e-02,
         2.17451001e-04, 4.79353158e-04, 5.90047485e-05, 1.74026474e-04,
         3.93739724e-04, 1.95872665e-04, 3.36651840e-04, 1.99999824e+00,
         1.99999458e+00, 1.99960088e+00, 4.14817519e-01, 3.75285192e-03,
         1.99998869e+00, 1.99999390e+00, 1.99999390e+00, 1.99864798e+00,
         1.99892192e+00, 1.99892194e+00, 1.28676056e-02, 1.59282213e-02,
         1.59278152e-02, 5.77945225e-01, 8.23806014e-01, 8.72295856e-01,
         5.77959987e-01, 9.69448112e-01, 6.88424180e-03, 1.27261860e-02,
         1.29830250e-02, 6.88415890e-03, 1.35037877

In [4]:
F = mf.get_fock()

In [151]:
import numpy as np
import scipy
import copy as cp
import math

def get_frag_bath(Pin, frag, S, thresh=1e-7, verbose=1):
    print(" Frag: ", frag)
    X = scipy.linalg.sqrtm(S)
    Xinv = scipy.linalg.inv(X)

    Nbas = S.shape[0]
    Cfrag = Xinv@np.eye(Nbas)[:,frag]
    

    P = X@Pin@X.T

    nfrag = np.trace(P[frag,:][:,frag])
    P[frag,:] = 0
    P[:,frag] = 0
    bath_idx = []
    env_idx = []
    e,U = np.linalg.eigh(P)
    nbath = 0.0
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            env_idx.append(nidx)
        elif thresh < ni < 1-thresh:
            if verbose > 1:
                print(" eigvalue: %12.8f" % ni)
            bath_idx.append(nidx)
            nbath += ni
        
            
    print(" # Electrons frag: %12.8f bath: %12.8f total: %12.8f" %(nfrag, nbath, nfrag+nbath))
    Cenv = Xinv@U[:,env_idx]
    Cbath = Xinv@U[:,bath_idx]
    
    
    C = np.hstack((Cfrag, Cbath))
    
    # Get virtual orbitals (these are just the orthogonal complement of the env and frag/bath
    if verbose > 1:
        print(" Now find span of virtual space")
    Q = np.eye(C.shape[0]) - X@C@C.T@X - X@Cenv@Cenv.T@X
    e,U = np.linalg.eigh(Q)
    vir_idx = []
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            vir_idx.append(nidx)
    Cvir = Xinv@U[:,vir_idx]

    assert(Cenv.shape[1] + Cvir.shape[1] + C.shape[1] == Nbas)

          
    # print(C.T@S@C)
    return (Cenv, C, Cvir)

def gram_schmidt(frags, S, thresh=1e-8):
    # |v'> = (1-sum_i |i><i|) |v>
    #      = |v> - sum_i |i><i|v>
    Nbas = S.shape[1]
    seen = []
    out = []
    seen = np.zeros((Nbas, 0))

    for f in frags:
        outf = np.zeros((Nbas, 0))
        # grab each orbital
        for fi in range(f.shape[1]):
            v = f[:,fi]
            v.shape = (Nbas, 1)

            # Compare to previous orbitals
            for fj in range(seen.shape[1]):
                j = seen[:,fj]
                j.shape = (Nbas, 1)
                ovlp = (j.T @ S @ v)[0]
                v = v - j * ovlp

                norm = np.sqrt((v.T @ S @ v)[0])
                if norm < thresh:
                    print(" Warning: small norm in GS: ", norm)
                v = v/norm
            
            outf = np.hstack((outf, v))
            seen = np.hstack((seen, v))
        out.append(outf)
    return out


In [167]:

# Find AO's corresponding to atoms 
# print(mf.mol.aoslice_by_atom())
# print(mf.mol.ao_labels(fmt=False, base=0))
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3s", "3p"):
            frag1.append(ao_idx)
            full.append(ao_idx)
        elif ao[2] in ("3d",):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3s", "3p"):
            frag3.append(ao_idx)
            full.append(ao_idx)
        if ao[2] in ("3d",):
            frag4.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3, frag4]
P = mf.make_rdm1()
Pa = P[0,:,:]
Pb = P[1,:,:]

C = mf.mo_coeff
S = mf.get_ovlp()

(Cenv, Cact, Cvir) = get_frag_bath(Pa, full, S)
pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", Cact)

CC = np.hstack((Cenv, Cact, Cvir))

# Since we used Pa for the orbital partitioning, we will have an integer number of alpha electrons,
# but not beta. however, we can determine number of alpha because the env is closed shell.
n_tot_b = mf.nelec[1]
na_act = np.trace(Cact.T @ S @ Pa @ S @ Cact)
na_env = np.trace(Cenv.T @ S @ Pa @ S @ Cenv)
na_vir = np.trace(Cvir.T @ S @ Pa @ S @ Cvir)
nb_env = na_env
nb_act = n_tot_b - nb_env
nb_vir = na_vir
print(" # electrons: %12s %12s" %("α", "β"))
print("         Env: %12.8f %12.8f" %(na_env, nb_env))
print("         Act: %12.8f %12.8f" %(na_act, nb_act))
print("         Vir: %12.8f %12.8f" %(na_vir, nb_vir))

pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", Cact)


Pa = Cact @ Cact.T @ S @ Pa @ S @ Cact @ Cact.T

(Ce1, Cf1, Cv1) = get_frag_bath(Pa, frag1, S)
(Ce2, Cf2, Cv2) = get_frag_bath(Pa, frag2, S)
(Ce3, Cf3, Cv3) = get_frag_bath(Pa, frag3, S)
(Ce4, Cf4, Cv4) = get_frag_bath(Pa, frag4, S)

frag_orbs = gram_schmidt((Cf1, Cf2, Cf3, Cf4), S)

Nbas = S.shape[1]
Ctot = np.zeros((Nbas,0))

clusters = []
init_fspace = []
for fi,f in enumerate(frag_orbs):
    Ctot = np.hstack((Ctot, f))
    Paf = f.T @ S @ Pa @ S @ f
    Pbf = f.T @ S @ Pb @ S @ f
    
    na = np.trace(Paf)
    nb = np.trace(Pbf)
    
    clusters.append(frags[fi])
    init_fspace.append((int(np.round(na)), int(np.round(nb))))
    print(" Fragment: %4i %s" %(fi,frags[fi]))
    print("    # α electrons: %12.8f" % na)
    print("    # β electrons: %12.8f" % nb)

    pyscf.tools.molden.from_mo(mf.mol, "C_frag%2i.molden"%fi, f)

pyscf.tools.molden.from_mo(mf.mol, "C_act.molden", Ctot)


 Frag:  [2, 8, 9, 10, 14, 15, 16, 17, 18, 33, 39, 40, 41, 45, 46, 47, 48, 49]
 # Electrons frag:  13.45548689 bath:   4.54451311 total:  18.00000000
 # electrons:            α            β
         Env:  51.00000000  51.00000000
         Act:  18.00000000  12.00000000
         Vir:   0.00000000   0.00000000
 Frag:  [2, 8, 9, 10]
 # Electrons frag:   3.90082843 bath:   0.09917157 total:   4.00000000
 Frag:  [14, 15, 16, 17, 18]
 # Electrons frag:   2.82691477 bath:   2.17308523 total:   5.00000000
 Frag:  [33, 39, 40, 41]
 # Electrons frag:   3.90082821 bath:   0.09917179 total:   4.00000000
 Frag:  [45, 46, 47, 48, 49]
 # Electrons frag:   2.82691548 bath:   2.17308452 total:   5.00000000
 Fragment:    0 [2, 8, 9, 10]
    # α electrons:   4.00000000
    # β electrons:   3.99978550
 Fragment:    1 [14, 15, 16, 17, 18]
    # α electrons:   5.00000000
    # β electrons:   2.00775255
 Fragment:    2 [33, 39, 40, 41]
    # α electrons:   4.00000000
    # β electrons:   3.99965790
 Fragment:

In [160]:
print(clusters)
print(init_fspace)

[[2, 8, 9, 10], [14, 15, 16, 17, 18], [33, 39, 40, 41], [45, 46, 47, 48, 49]]
[(4, 4), (5, 2), (4, 4), (5, 2)]


# Make Integrals

In [162]:
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

In [168]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Ctot.T @ h @ Ctot;
j = Ctot.T @ j @ Ctot;
k = Ctot.T @ k @ Ctot;

In [169]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Ctot, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [170]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Ctot @ Ctot.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [172]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Ctot)
np.save("overlap_mat", S)